In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_parquet("data/driver_stats.parquet")

In [3]:
df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2023-11-21 08:00:00+00:00,1005,0.335094,0.162989,351,2023-12-06 08:57:13.838
1,2023-11-21 09:00:00+00:00,1005,0.100321,0.105751,961,2023-12-06 08:57:13.838
2,2023-11-21 10:00:00+00:00,1005,0.664010,0.070998,780,2023-12-06 08:57:13.838
3,2023-11-21 11:00:00+00:00,1005,0.259892,0.082264,705,2023-12-06 08:57:13.838
4,2023-11-21 12:00:00+00:00,1005,0.505575,0.898033,830,2023-12-06 08:57:13.838


In [4]:
df.shape

(1807, 6)